# 뉴스 데이터 xml 수집하기 - sbs news

In [1]:
import requests
from bs4 import BeautifulSoup

news_rss = requests.get('https://news.sbs.co.kr/news/SectionRssFeed.do?sectionId=07')
news_rss_soup = BeautifulSoup(news_rss.content, 'xml')
link_list = news_rss_soup.select('item > link')
print("기사 개수:", len(link_list))

기사 개수: 29


In [2]:
title_list = news_rss_soup.select('item > title')
title_list = [title.text for title in title_list]
print(title_list)

['중국 드론 업체 DJI "증거 없이 수입 금지"…미국 법원에 소송', '미국, 인도·인도네시아 태양광 고율 관세…한화, 반사 이익?', '미국 최대 은행 JP모건도 AI발 대규모 인력 재배치 계획', '영국, 입국 전 전자여행허가 확대…한국 포함 85개국에 의무화', "'상호관세 돌려달라' 소송 봇물…로레알·다이슨도 합류", '멕시코 마약왕 사살 혼란에…혼다, 현지 공장 가동중단 뒤 재개', '술 먹는 침팬지…"야생 소변 샘플 20개 검사, 17개서 알코올 대사물 검출"', '미국, 유럽·중동에 군용기 150대 이동…이라크전 이후 최대', '미국, 서안지구서 첫 영사 서비스…이스라엘 "환영"', '이란, 미국과 핵협상 앞두고 "타결 가시권…외교 최우선해야"', '미국 법원, "오픈AI가 영업비밀 빼갔다" 주장한 xAI 소송 기각', '중국 누리꾼, "한국은 문화 도둑국" "중국설 훔쳤다" 주장', '길거리서 무료로…"참된 스승" 쏟아진 찬사', "스키 타다 '화들짝'…설원 가로질러 전력 질주", '노인 도왔더니 "4,500만 원 배상하라"…논란 터진 장면', '유엔총회, 우크라 지지 결의 채택…미·중은 기권', '오늘 국정연설…"백악관, 관세 15%로 인상 작업 중"', 'FBI국장이 왜 거기서 나와…미 하키팀 금메달 뒤풀이 참석 구설', '"미국, 은행에 고객 시민권정보 수집 요구 검토…이민단속 일환"', '보석 도둑맞은 루브르 박물관장 끝내 사임…마크롱 수락', '러 매체 "한국학자 란코프 국민대 교수, 라트비아서 체포"', '미국 유명앵커, 모친 실종 3주 만에 현상금 14억 내걸며 호소', '푸틴 "적들이 러시아 패배 위해 모든 방법 찾아…후회할 것"', '브라질 남동부 폭우 강타…"사망·실종자 60여 명"', '월드컵 \'한국 경기\' 도시, 치안 악화에 멕시코 대통령 "안전 보장"', '백악관 당국자 "글로벌 관세, 15%로 올리는 작업 진행중"', '백악관 "트럼프 첫 옵션은 외교…필요하면 이란 공격"', '스위스, 베네수 마두로 측근 등 자산

In [3]:
news_data = []
for link in link_list:
    news_response = requests.get(link.text)
    news_content_soup = BeautifulSoup(news_response.content, 'html.parser')
    news_content = news_content_soup.select_one("div[itemprop=articleBody]")
    news_data.append(news_content.text)

import pandas as pd
news_df = pd.DataFrame(data={'title': title_list, 'content': news_data})
print(news_df.head())

news_df.to_csv("news.csv", encoding="utf-8-sig", index=False)
print("Save complete")

                                  title  \
0  중국 드론 업체 DJI "증거 없이 수입 금지"…미국 법원에 소송   
1     미국, 인도·인도네시아 태양광 고율 관세…한화, 반사 이익?   
2      미국 최대 은행 JP모건도 AI발 대규모 인력 재배치 계획   
3    영국, 입국 전 전자여행허가 확대…한국 포함 85개국에 의무화   
4         '상호관세 돌려달라' 소송 봇물…로레알·다이슨도 합류   

                                             content  
0  \n\n▲ 미국 상점에 전시된 DJI 무인기\n\n 외국산 신형 무인기(드론) 및 ...  
1  \n\n▲ 태양광 에너지\n\n 미국 정부가 인도와 인도네시아 등지에서 생산되는 태...  
2  \n\n▲ 제이미 다이먼 JP모건 회장\n\n 미국 최대 은행인 JP모건체이스의 제...  
3  \n\n▲ 영국의 한 공항\n\n 영국이 25일(현지시간)부터 무비자 입국이 가능한...  
4  \n\n▲ 미국 연방 대법원\n\n 미국 연방 대법원이 최근 위법으로 판결한 도널드...  
Save complete


# 뉴스 컨텐츠 클린징

In [4]:
news_data_1 = []
for link in link_list:
    news_response = requests.get(link.text)
    news_content_soup = BeautifulSoup(news_response.content, 'html.parser')

    news_content = news_content_soup.select_one("div[itemprop=articleBody]")

    if news_content is not None:
          news_data_1.append(news_content.text.strip())
    else:
        news_data_1.append("본문 없음")  # 또는 None, ""
news_data_1_df = pd.DataFrame(data={'title': title_list, 'content': news_data_1})

In [5]:
news_data_1_df.head()

,title,content
0,"중국 드론 업체 DJI ""증거 없이 수입 금지""…미국 법원에 소송",▲ 미국 상점에 전시된 DJI 무인기\n\n 외국산 신형 무인기(드론) 및 관련 부...
1,"미국, 인도·인도네시아 태양광 고율 관세…한화, 반사 이익?",▲ 태양광 에너지\n\n 미국 정부가 인도와 인도네시아 등지에서 생산되는 태양광 발...
2,미국 최대 은행 JP모건도 AI발 대규모 인력 재배치 계획,▲ 제이미 다이먼 JP모건 회장\n\n 미국 최대 은행인 JP모건체이스의 제이미 다...
3,"영국, 입국 전 전자여행허가 확대…한국 포함 85개국에 의무화","▲ 영국의 한 공항\n\n 영국이 25일(현지시간)부터 무비자 입국이 가능한 한국,..."
4,'상호관세 돌려달라' 소송 봇물…로레알·다이슨도 합류,▲ 미국 연방 대법원\n\n 미국 연방 대법원이 최근 위법으로 판결한 도널드 트럼프...


In [6]:
missing_count = sum(1 for content in news_data_1 if content == "본문 없음")
print(f"본문이 없는 기사 개수: {missing_count}개")
print(f"전체 기사 개수: {len(news_data_1)}개")

본문이 없는 기사 개수: 0개
전체 기사 개수: 29개


In [7]:
import pandas as pd

news_df = pd.DataFrame({
    'title': title_list,
    'content': news_data
})
news_df['is_missing'] = news_df['content'] == "본문 없음"
missing_count = news_df['is_missing'].sum()
total_count = len(news_df)

print(f"본문 없음: {missing_count}개 / 전체: {total_count}개 ({missing_count/total_count:.1%})")

본문 없음: 0개 / 전체: 29개 (0.0%)
